<img src="./University_Debrecen_logo.jpg" alt="Drawing" style="width: 200px;"/>

# Analysis of the recordings containing SIMV-VG-PS ventilation mode

### 1. Import the required modules and set options

In [ ]:
import pickle
import sys
import copy
import os
import itertools
from collections import defaultdict
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib as mpl
import scipy as sp
import matplotlib.pyplot as plt
from pandas import DataFrame, Series
from matplotlib import dates
from scipy import stats

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)
pd.options.mode.chained_assignment = None
majorFmt = dates.DateFormatter('%d/%m\n%H:%M')

In [ ]:
print(f'Python version: {sys.version}')
print(f'pandas version: {pd.__version__}')
print(f'matplotlib version: {mpl.__version__}')
print(f'numPy version: {np.__version__}')
print(f'scipy version: {sp.__version__}')

### 2. List and set the working directory and the directory to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'SIMV_VG_PS'

# Name of the external hard drives
DRIVE_IN = os.path.join(os.sep, 'Users', 'guszti', 'Library', 'Mobile Documents', 'com~apple~CloudDocs', 
                                'Documents', 'Research', 'Ventilation')
DRIVE_OUT = 'Guszti'

# Directory containing clinical and blood gas data data
DIR_READ_CLIN = os.path.join(DRIVE_IN, 'ventilation_draeger_debrecen')

# Folder on external drive to read the ventilation data from
DIR_READ_VENT =  os.path.join(os.sep, 'Volumes', DRIVE_OUT, 'draeger_debrecen',)

# Folder to write statistics and reports on the group
DIR_WRITE = os.path.join(DRIVE_IN, 'ventilation_draeger_debrecen', 'Analyses', TOPIC)
os.makedirs(DIR_WRITE, exist_ok = True)

# Folder on external drive to export graphs and data about individual recordings
DATA_DUMP = os.path.join(os.sep, 'Volumes', DRIVE_OUT, 'data_dump', 'draeger_debrecen', TOPIC)
os.makedirs(DATA_DUMP, exist_ok = True)

In [ ]:
DIR_READ_CLIN, DIR_READ_VENT, DIR_WRITE, DATA_DUMP

### 3. Import processed data 

#### A. Ventilation modes

In [ ]:
with open(os.path.join(DATA_DUMP, 'has_simv_frme.pickle'), 'rb') as handle:
    has_simv_frme  = pickle.load(handle)

In [ ]:
has_simv_frme.head()

In [ ]:
len(has_simv_frme)

#### B. Ventilator data from HDF5 archives

In [ ]:
%%time

data = defaultdict(dict)
for patient, recording in has_simv_frme.index:
    print(f'Importing {patient} {recording}')
    data[patient][recording] = pd.read_hdf(os.path.join(DATA_DUMP, f'{patient}_{recording}_pars_and_settings.h5'), 
            key = patient)

In [ ]:
len(data)

#### C. Import processed clinical details

This recording list is produced by the `Clinical_details_processing_debrecen.ipynb` notebook

In [ ]:
clinical_details_recordings = pd.read_csv(os.path.join(DIR_READ_CLIN, 'clinical_details_recordings.csv'))
# Only keep recordings until LVD020
clinical_details_recordings = \
    clinical_details_recordings[clinical_details_recordings['Patient'].apply(lambda x: int(x[3:].lstrip()) <=20)]

clinical_details_recordings;

#### D. Carbon dioxide data
This recording list is produced by the `Carbon_dioxide data_processing_debrecen.ipynb` notebook

In [ ]:
with open(os.path.join(DATA_DUMP, 'blood_gases_processed.pickle'), 'rb') as handle:
    blood_gases = pickle.load(handle)

# Only keep recordings until LVD020    
blood_gases = {key:value for key, value in blood_gases.items() if int(key[3:].lstrip()) <=20}

blood_gases.keys()

In [ ]:
blood_gases['LVD003'].info()

In [ ]:
with open(os.path.join(DATA_DUMP, 'transcutaneous_data_processed.pickle'), 'rb') as handle:
    transcutaneous = pickle.load(handle)

# Only keep recordings until LVD020
transcutaneous  = {key:value for key, value in transcutaneous.items() if int(key[3:].lstrip()) <=20}

In [ ]:
transcutaneous.keys()

In [ ]:
transcutaneous['LVD003'].info()

In [ ]:
transcutaneous['LVD003'].head()

### 4. Clean up ventilator data

Identify and remove abnormal inflations (presumaby ventilation artifacts)

In [ ]:
data_points_before_cleanup = {}
for patient in data:
    for recording in data[patient]:
        data_points_before_cleanup[(patient, recording)] = len(data[patient][recording])

In [ ]:
data_points_before_cleanup

#### Inflations with negative inflating pressure
These are artifacts

In [ ]:
%%time

Pinfl_negative = {}
for patient in data:
    for recording in data[patient]:
        Pinfl_negative[(patient, recording)] = sum(data[patient][recording]['Pinfl_mand [mbar]'] <0)

In [ ]:
Pinfl_negative

#### Inflations with VTmand or VTspon  <0.1 or  >25 mL/kg

VT > 25 mL/kg is likely abnormal,  representing a flow sensor problem or an open circuit.

In [ ]:
%%time

VTmand_over25 = {}
VTspon_over25 = {}
for patient in data:
    for recording in data[patient]:
        if 'VTmand [mL/kg]' in data[patient][recording].columns:
            VTmand_over25[(patient, recording)] = (sum(data[patient][recording]['VTmand [mL/kg]'] > 25))
        if 'VTspon [mL/kg]' in data[patient][recording].columns:
            VTspon_over25[(patient, recording)] = (sum(data[patient][recording]['VTspon [mL/kg]'] > 25))

In [ ]:
VTmand_over25

In [ ]:
VTspon_over25

#### Inflations with MV over 1 L/min/kg

This will not occur during conventional ventilation, unless due to an artifact (eg circuit is open). 

In [ ]:
%%time

MV_over_1 = {}

for patient in data:
    for recording in data[patient]:
        if 'MV [L/min/kg]' in data[patient][recording].columns:
            MV_over_1[(patient, recording)] = sum(data[patient][recording]['MV [L/min/kg]'] > 1) 

In [ ]:
MV_over_1

#### Inflations with RR  > 130/min

These values are likely abnormal representing autocycling due to condensed water in the circuit

In [ ]:
%%time

RR_over_130 = {}

for patient in data:
    for recording in data[patient]:
        if 'RR [1/min]' in data[patient][recording].columns:
            RR_over_130[(patient, recording)] = sum(data[patient][recording]['RR [1/min]'] > 130) 

In [ ]:
RR_over_130

#### Remove these anomalies from the dataset

With this step missing data (nan) are also removed

In [ ]:
%%time

for patient in data:
    #print(patient)
    for recording in data[patient]:
        data[patient][recording] = data[patient][recording][data[patient][recording]['Pinfl_mand [mbar]'] >= 0]
        data[patient][recording] = data[patient][recording][data[patient][recording]['VTmand [mL/kg]'] <= 25]    
        data[patient][recording] = data[patient][recording][data[patient][recording]['VTspon [mL/kg]'] <= 25]   
        data[patient][recording] = data[patient][recording][data[patient][recording]['MV [L/min/kg]'] <= 1]
        data[patient][recording] = data[patient][recording][data[patient][recording]['RR [1/min]'] <= 130]

In [ ]:
data_points_after_cleanup = {}
for patient in data:
    for recording in data[patient]:
        data_points_after_cleanup[(patient, recording)] = len(data[patient][recording])

In [ ]:
cleanup_frme = DataFrame([data_points_before_cleanup, data_points_after_cleanup, Pinfl_negative, 
                          VTmand_over25, VTspon_over25, MV_over_1, RR_over_130]).T
cleanup_frme.columns =   ['data_points_before_cleanup', 'data_points_after_cleanup', 'Pinfl_negative', 
                          'VTmand_over_25', 'VTspon_over_25', 'MV_over_1', 'RR_over_130',]
cleanup_frme['rows_removed'] = cleanup_frme['data_points_before_cleanup'] - \
                                cleanup_frme['data_points_after_cleanup']

In [ ]:
cleanup_frme.sum()

In [ ]:
cleanup_frme.sum()/ cleanup_frme['data_points_before_cleanup'].sum()

In [ ]:
cleanup_frme.to_csv(os.path.join(DIR_WRITE, f'data_cleanup.csv'))

writer = pd.ExcelWriter(os.path.join(DIR_WRITE, f'data_cleanup.xlsx'))
cleanup_frme.to_excel(writer, 'cleanup_frme')
writer.save()

In [ ]:
qc = cleanup_frme.reset_index()
ind_1 = [pat for pat, _ in qc['index']]
ind_2 = [rec for _ , rec in qc['index']]
qc.index = [ind_1, ind_2]
# Combine recordings belonging to the same patient
qc = qc.groupby(level=0).sum()
qc

In [ ]:
qc.to_csv(os.path.join(DIR_WRITE, f'qc.csv'))

writer = pd.ExcelWriter(os.path.join(DIR_WRITE, f'qc.xlsx'))
qc.to_excel(writer, 'qc')
writer.save()

### 5. Combine data from the same patient

In [ ]:
%%time

data_combined = {}
for patient in data:
    #print(patient)
    data_combined[patient] = pd.concat(data[patient])

### 6. Select only recordings and recording parts with leak compensation and volume guarantee on

#### A. Generate DataFrame how frequently was leak compensation and VG `on` or `off`

In [ ]:
%%time

stat_modes = {}
for patient in data_combined:
    stat_modes[patient] = {'VG on': sum(data_combined[patient]['VG'] == 'on'),
                           'VG off': sum(data_combined[patient]['VG'] == 'off'),
                           'leak compensation on': sum(data_combined[patient]['leak compensation'] == 'on'),
                           'leak compensation off': sum(data_combined[patient]['leak compensation'] == 'off')}
    
stat_modes = DataFrame(stat_modes).T
stat_modes

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'stat_modes.xlsx'))
for patient in stat_modes:
    stat_modes[patient].to_excel(writer, patient)
writer.save()

##### B. Remove recordings or periods when leak compensation was off

In `LVD001` and in part of `LVD016`l eak compensation was off , do not include these parts in selection

In [ ]:
%%time

data_combined_selected = {}

for patient in data_combined:
    data_combined_selected[patient] = \
        data_combined[patient][data_combined[patient]['leak compensation'] == 'on'] 

In [ ]:
for patient in sorted(data_combined_selected.keys()):
    if data_combined_selected[patient].empty:
        del data_combined_selected[patient]

In [ ]:
data_combined_selected.keys()

##### C. Remove recordings or periods when volume guarantee was off

During part of `LVD010` VG was off. Remove it.

In [ ]:
%%time

for patient in data_combined_selected:
    data_combined_selected[patient] = \
        data_combined_selected[patient][data_combined_selected[patient]['VG'] == 'on'] 

In [ ]:
for patient in sorted(data_combined_selected.keys()):
    if data_combined_selected[patient].empty:
        del data_combined_selected[patient]

In [ ]:
data_combined_selected.keys()

### 7. Exclude recordings which were shorter than 12 hours

In [ ]:
recording_duration = {}

for patient in data_combined_selected:
    recording_duration[patient] = len(data_combined_selected[patient])

recording_duration = DataFrame(Series(recording_duration), columns = ['recording duration [s]'])
recording_duration['recording duration [hours]'] = recording_duration['recording duration [s]'] / 3600
recording_duration['recording duration [days]'] = recording_duration['recording duration [s]'] / (3600 * 24)
recording_duration;

In [ ]:
recording_duration_selected = recording_duration[recording_duration['recording duration [hours]'] > 12]
len(recording_duration_selected)

In [ ]:
recording_duration_selected

In [ ]:
recording_duration_selected.describe()

In [ ]:
recording_duration_selected.sum()

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'recording_duration_selected.xlsx'))
recording_duration_selected.to_excel(writer, 'rec_duration_selected')
recording_duration_selected.describe().to_excel(writer, 'stats')
recording_duration_selected.sum().to_excel(writer, 'total')
writer.save()

#### Limit ventilator recordings to those ones longer that 12 hours

In [ ]:
data_combined_selected = {key:value for key, value in data_combined_selected.items() 
                         if key in recording_duration_selected.index}
len(data_combined_selected)

### 8. Perform descriptive statistics on the clinical details of the final dataset

In [ ]:
clinical_details_recordings = pd.read_csv(os.path.join(DIR_WRITE, 'clinical_details_recordings_simv.csv'))
clinical_details_recordings = clinical_details_recordings.reset_index()
clinical_details_recordings = clinical_details_recordings.drop_duplicates(subset=['Patient'], keep='first')
clinical_details_recordings = clinical_details_recordings.set_index('Patient', drop=True)

columns_to_keep = ['Gestation at birth', 'Date of birth', 'Birth weight', 'Current weight', 'Main diagnoses',
                   'Mode of intubation', 'ETT size', 'ETT insertion length', 'Comment',
                   'Recording start', 'Postnatal age', 'Corrected gestation']

clinical_details_recordings = clinical_details_recordings[columns_to_keep]
clinical_details_recordings.head()

In [ ]:
clinical_details_selected = clinical_details_recordings.reindex(data_combined_selected.keys())
clinical_details_selected

In [ ]:
clinical_details_recordings.info()

In [ ]:
clinical_details_selected.describe()

In [ ]:
sum(clinical_details_selected['Birth weight'] >= 2000)

In [ ]:
sum(clinical_details_selected['Birth weight'] < 2000)

In [ ]:
sum(clinical_details_selected['Birth weight'] < 1000)

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'clinical_details_stats.xlsx'))
clinical_details_selected.to_excel(writer, 'clinical_details')
clinical_details_selected.describe().to_excel(writer, 'stats')
writer.save()

### 9. Generate `mean` values of each parameter over 1-minute periods, 10-minute periods and over 1-hours

In [ ]:
%%time

# Over 1-minute periods
data_combined_selected_1min_mean = {}
for patient in data_combined_selected:
    # The label is the beginning of the period
    data_combined_selected_1min_mean[patient] = \
        data_combined_selected[patient].reset_index(0).resample('1min', label='left', closed='left').mean()
    
# Over 10-minute periods
data_combined_selected_10min_mean = {}
for patient in data_combined_selected:
    # The label is the beginning of the period
    data_combined_selected_10min_mean[patient] = \
        data_combined_selected[patient].reset_index(0).resample('10min', label='left', closed='left').mean()
    
# Over 1-hour periods
data_combined_selected_1hour_mean = {}
for patient in data_combined_selected:
    # The label is the beginning of the period
    data_combined_selected_1hour_mean[patient] = \
        data_combined_selected[patient].reset_index(0).resample('1H', label='left', closed='left').mean()

### 10. Combine 10-minute and 1-hour aggregated data with transcutaneous CO2 data and blood gases

##### Aggregate over 10 minutes

In [ ]:
data_combined_selected_10min_mean_with_CO2 = {}
for patient in data_combined_selected_10min_mean:
    if patient in transcutaneous:
        data_combined_selected_10min_mean_with_CO2[patient] = pd.merge(data_combined_selected_10min_mean[patient],
            transcutaneous[patient]['tcCO2'], left_index=True, right_index=True, how='inner')

In [ ]:
for patient in data_combined_selected_10min_mean_with_CO2:
    print(patient, len(data_combined_selected_10min_mean_with_CO2[patient]))

#### Aggregate over 1-hour periods

Use the mean tcCO2 over the last 15 minutes of the recordings

In [ ]:
transcutaneous_1hour = {}
for patient in transcutaneous:
    # Use the last data (mean over the last 1o minutes of the hour periods, unless it is NA 
    # in which case the previous one is used)
    # The label still comes from the beginning of the period, as for ventilator data
    transcutaneous_1hour[patient] = \
        transcutaneous[patient][['tcCO2']].dropna().resample('1H', label='left', closed='right').last()

In [ ]:
data_combined_selected_1hour_mean_with_CO2 = {}
for patient in data_combined_selected_1hour_mean:
    if patient in transcutaneous:
        data_combined_selected_1hour_mean_with_CO2[patient] = pd.merge(data_combined_selected_1hour_mean[patient],
            transcutaneous_1hour[patient]['tcCO2'], left_index=True, right_index=True, how='inner')

In [ ]:
for patient in data_combined_selected_1hour_mean_with_CO2:
    print(patient, len(data_combined_selected_1hour_mean_with_CO2[patient]))

### 11. Identify and remove those periods when there was no spontaneous breathing

#### How frequently is the mean VTspon / RRspon / MVespon ) zero over the 1-minute periods

In [ ]:
for patient in data_combined_selected_1min_mean:
    data = data_combined_selected_1min_mean[patient]
    filt = data[data['VTspon [mL/kg]'] == 0]
    filt_2 = data[data['MVespon [L/min/kg]'] == 0]
    filt_3 = data[data['RRspon [1/min]'] == 0]
    filt_4 = filt[filt['RRspon [1/min]'] == 0]
    print(patient, len(data), len(filt), len(filt_2), len(filt_3), len(filt_4))

In [ ]:
spont_breathing_1min = {}
for patient, data in data_combined_selected_1min_mean.items():
    spont_breathing_1min[patient] = {'total_periods' : len(data), 
                                     'periods_with_spont_breathing' : len(data[data['VTspon [mL/kg]'] > 0])}   
spont_breathing_1min = DataFrame(spont_breathing_1min).T

spont_breathing_10min = {}
for patient, data in data_combined_selected_10min_mean.items():
    spont_breathing_10min[patient] = {'total_periods' : len(data), 
                                     'periods_with_spont_breathing' : len(data[data['VTspon [mL/kg]'] > 0])}
spont_breathing_10min = DataFrame(spont_breathing_10min).T

spont_breathing_1hour = {}
for patient, data in data_combined_selected_1hour_mean.items():
    spont_breathing_1hour[patient] = {'total_periods' : len(data), 
                                     'periods_with_spont_breathing' : len(data[data['VTspon [mL/kg]'] > 0])}
spont_breathing_1hour = DataFrame(spont_breathing_1hour).T

spont_breathing = pd.concat([spont_breathing_1min, spont_breathing_10min, spont_breathing_1hour], axis=1,
                           keys=['1_minute', '10_minute', '1_hour'])
spont_breathing

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'periods_with_spont_breathing.xlsx'))
spont_breathing.to_excel(writer)
writer.save()

#### Remove periods with no spontaneous breathing

In [ ]:
%%time

# Only consider those minutes when mean VTspon was not 0 (= there were spontaneous breaths)
for patient, data in data_combined_selected_1min_mean.items():
    data_combined_selected_1min_mean[patient] = data[data['VTspon [mL/kg]'] > 0]
    
for patient, data in data_combined_selected_10min_mean.items():
    data_combined_selected_10min_mean[patient] = data[data['VTspon [mL/kg]'] > 0]
    
for patient, data in data_combined_selected_1hour_mean.items():
    data_combined_selected_1hour_mean[patient] = data[data['VTspon [mL/kg]'] > 0]
    
    
for patient, data in data_combined_selected_10min_mean_with_CO2.items():
    data_combined_selected_10min_mean_with_CO2[patient] = data[data['VTspon [mL/kg]'] > 0]
    
for patient, data in data_combined_selected_1hour_mean_with_CO2.items():
    data_combined_selected_1hour_mean_with_CO2[patient] = data[data['VTspon [mL/kg]'] > 0]

### 12. Generate fields with the ratios of PIPspon/mand, VTspon/mand, RRspon/mand, MVspon/mand, Tispon_mand

In [ ]:
%%time

for patient, data in data_combined_selected_1min_mean.items():
    data['PIP_spon_mand'] = data['PIP_spon [mbar]'] / data['PIP [mbar]']
    data['VT_spon_mand'] = data['VTspon [mL/kg]'] / data['VTmand [mL/kg]']
    data['RR_spon_mand'] = data['RRspon [1/min]'] / data['RRmand [1/min]']
    data['MVe_spon_mand'] = data['MVespon [L/min/kg]'] / data['MVemand [L/min/kg]']
    data['Ti_spon_mand'] = data['Tispon [s]'] / data['Ti_set [s]']
    
for patient, data in data_combined_selected_10min_mean.items():
    data['PIP_spon_mand'] = data['PIP_spon [mbar]'] / data['PIP [mbar]']
    data['VT_spon_mand'] = data['VTspon [mL/kg]'] / data['VTmand [mL/kg]']
    data['RR_spon_mand'] = data['RRspon [1/min]'] / data['RRmand [1/min]']
    data['MVe_spon_mand'] = data['MVespon [L/min/kg]'] / data['MVemand [L/min/kg]']
    data['Ti_spon_mand'] = data['Tispon [s]'] / data['Ti_set [s]']     

for patient, data in data_combined_selected_1hour_mean.items():
    data['PIP_spon_mand'] = data['PIP_spon [mbar]'] / data['PIP [mbar]']
    data['VT_spon_mand'] = data['VTspon [mL/kg]'] / data['VTmand [mL/kg]']
    data['RR_spon_mand'] = data['RRspon [1/min]'] / data['RRmand [1/min]']
    data['MVe_spon_mand'] = data['MVespon [L/min/kg]'] / data['MVemand [L/min/kg]']
    data['Ti_spon_mand'] = data['Tispon [s]'] / data['Ti_set [s]']
    
for patient, data in data_combined_selected_10min_mean_with_CO2.items():
    data['PIP_spon_mand'] = data['PIP_spon [mbar]'] / data['PIP [mbar]']
    data['VT_spon_mand'] = data['VTspon [mL/kg]'] / data['VTmand [mL/kg]']
    data['RR_spon_mand'] = data['RRspon [1/min]'] / data['RRmand [1/min]']
    data['MVe_spon_mand'] = data['MVespon [L/min/kg]'] / data['MVemand [L/min/kg]']
    data['Ti_spon_mand'] = data['Tispon [s]'] / data['Ti_set [s]']     

for patient, data in data_combined_selected_1hour_mean_with_CO2.items():
    data['PIP_spon_mand'] = data['PIP_spon [mbar]'] / data['PIP [mbar]']
    data['VT_spon_mand'] = data['VTspon [mL/kg]'] / data['VTmand [mL/kg]']
    data['RR_spon_mand'] = data['RRspon [1/min]'] / data['RRmand [1/min]']
    data['MVe_spon_mand'] = data['MVespon [L/min/kg]'] / data['MVemand [L/min/kg]']
    data['Ti_spon_mand'] = data['Tispon [s]'] / data['Ti_set [s]']

### 13. Combine DataFrames into one

In [ ]:
data_combined_selected_1min_mean_all = pd.concat(data_combined_selected_1min_mean).swaplevel()
data_combined_selected_10min_mean_all = pd.concat(data_combined_selected_10min_mean).swaplevel()
data_combined_selected_1hour_mean_all = pd.concat(data_combined_selected_1hour_mean).swaplevel()

data_combined_selected_10min_mean_with_CO2_all = pd.concat(data_combined_selected_10min_mean_with_CO2).swaplevel()
data_combined_selected_1hour_mean_with_CO2_all = pd.concat(data_combined_selected_1hour_mean_with_CO2).swaplevel()

In [ ]:
data_combined_selected_1min_mean_all.shape, data_combined_selected_10min_mean_all.shape, \
data_combined_selected_1hour_mean_all.shape, 

In [ ]:
data_combined_selected_10min_mean_with_CO2_all.shape, data_combined_selected_1hour_mean_with_CO2_all.shape, 

In [ ]:
data_combined_selected_1min_mean_all.head()

### 14. Visualise and analyse relationshop between PIP, VT, RR, MV and Ti of mandatory ventilator inflations and spontaneous breaths

In [ ]:
def iqr(x):
    return np.round(np.percentile(x, 25), 2), np.round(np.percentile(x, 75), 2)

#### A. `peak inspiratory pressure` of mandatory and spontaneous breaths

In [ ]:
%%time

dpi=200
filetype = 'pdf'

data = data_combined_selected_1min_mean

for patient in data:
    print(f'Working on {patient}')
    fig = plt.figure()
    fig.set_size_inches(16, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)
    
    data[patient]['PIP [mbar]'].plot(ax = ax, color = 'red',  marker='o', 
                                     linestyle='', markersize=1, x_compat = True)
    data[patient]['PIP_spon [mbar]'].plot(ax = ax, color = 'green', marker='o', 
                                     linestyle='', markersize=1, x_compat = True)
        
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.set_ylim(0, ax.get_ylim()[1])
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title(patient,  size = 12, color = 'black')
    ax.legend(['PIP_mand', 'PIP_spon',], ncol = 2, fontsize = 12, loc = 'best', markerscale=6)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
    fig.savefig(os.path.join(DATA_DUMP, f'{patient}_PIP_mand_spon_1min_mean.{filetype}'), 
        dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)
    if patient != 'LVD002':
        plt.close()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 200
filetype = 'pdf'

data = data_combined_selected_1min_mean_all.dropna(how='any', subset = ['PIP [mbar]', 'PIP_spon [mbar]'])
corr = stats.pearsonr(data['PIP [mbar]'], data['PIP_spon [mbar]'])

ax.scatter(data['PIP [mbar]'], data['PIP_spon [mbar]'], marker = 'o', s=10, facecolor = 'black', 
              edgecolor = 'blue', alpha = 0.05)
ax.text(10, 18, 'r: %.2f' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, 60)
ax.set_ylim(-1, 20)
ax.set_xlabel('PIPmand [mbar]', size = 14)
ax.set_ylabel('PIPspon [mbar]', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig(os.path.join(DIR_WRITE, f'scatterplot_PIPmand_PIPspon_1min.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi=200
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (16,4))

data = list(data_combined_selected_1min_mean_all['PIP_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

ax.boxplot(data_for_graph, whis = [5,95], patch_artist = True, showfliers = False,
    showmeans = True, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 2)
ax.hlines(0.66, *ax.get_xlim(), color='red', linestyle='--')
ax.set_xticklabels(xticklabels)
ax.set_ylabel('PIPspon / PIPmand')
    
fig.savefig(os.path.join(DIR_WRITE, f'PIP_spon_mand_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

In [ ]:
data = data_combined_selected_1min_mean_all.dropna(how='any', subset=['PIP [mbar]', 'PIP_spon [mbar]']).groupby(level=1)

PIP_mand = pd.concat([data.size(), data['PIP [mbar]'].median(), data['PIP [mbar]'].apply(iqr)], axis=1)
PIP_mand = np.round(PIP_mand, 1)
PIP_mand.columns = ['number_of_minutes', 'median', 'IQR']

PIP_spon = pd.concat([data.size(), data['PIP_spon [mbar]'].median(), data['PIP_spon [mbar]'].apply(iqr)], axis=1)
PIP_spon = np.round(PIP_spon, 1)
PIP_spon.columns = ['number_of_minutes', 'median', 'IQR']

PIP_spon_mand = pd.concat([data.size(), data['PIP_spon_mand'].median(), data['PIP_spon_mand'].apply(iqr)], axis=1)
PIP_spon_mand = np.round(PIP_spon_mand, 1)
PIP_spon_mand.columns = ['number_of_minutes', 'median', 'IQR']

PIP = pd.concat([PIP_mand, PIP_spon, PIP_spon_mand], axis=1)
PIP.columns = [['PIP_mand', 'PIP_mand', 'PIP_mand', 
                'PIP_spon', 'PIP_spon', 'PIP_spon',
                'PIP_spon_mand', 'PIP_spon_mand', 'PIP_spon_mand'], PIP.columns]

In [ ]:
PIP

In [ ]:
np.round(PIP.describe(), 2)

#### B. `Leak-compensated expired tidal volume` of the spontaneous and mandatory breaths

In [ ]:
%%time

dpi=200
filetype = 'pdf'

for patient in data_combined_selected_1min_mean:
    data = data_combined_selected_1min_mean[patient]
    print(f'Working on {patient}')
    
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)
    
    data['VTspon [mL/kg]'].plot(ax = ax, color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
    data['VTmand [mL/kg]'].plot(ax = ax, color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
    data['VT_set [mL/kg]'].plot(ax = ax, color = 'black', linewidth=2, linestyle='dashed', x_compat = True)
            
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.set_ylim(-2, 25)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title(patient,  size = 12, color = 'black')
    ax.legend(['VTspon', 'VTmand', 'VTset'], ncol = 3, fontsize = 12, loc = 'best', markerscale=6)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
    fig.savefig(os.path.join(DATA_DUMP, f'{patient}_VT_mand_spon_1min_mean.{filetype}'), 
                dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)
    if patient != 'LVD002':
        plt.close()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 200
filetype = 'pdf'

data = data_combined_selected_1min_mean_all.dropna(how='any', subset = ['VTmand [mL/kg]', 'VTspon [mL/kg]'])
corr = stats.pearsonr(data['VTmand [mL/kg]'], data['VTspon [mL/kg]'])

ax.scatter(data['VTmand [mL/kg]'], data['VTspon [mL/kg]'], 
           marker = 'o', s=10, facecolor = 'black', edgecolor = 'blue', alpha = 0.005)
ax.text(2, 13, 'r: %.2f' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, 15)
ax.set_ylim(-1, 15)
ax.set_xlabel('VTmand [mL/kg]', size = 14)
ax.set_ylabel('VTspon [mL/kg]', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig(os.path.join(DIR_WRITE, f'scatterplot_VTmand_VTspon.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi=200
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (16,4))

data = data_combined_selected_1min_mean_all
data = list(data['VT_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

ax.boxplot(data_for_graph, whis = [5,95], patch_artist = True, showfliers = False,
    showmeans = True, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_xticklabels(xticklabels)
ax.set_ylabel('VTspon / VTmand')
    
fig.savefig(os.path.join(DIR_WRITE, f'VT_spon_mand_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

In [ ]:
data = data_combined_selected_1min_mean_all.dropna(subset=['VTmand [mL/kg]', 'VTspon [mL/kg]']).groupby(level=1)

VT_mand = pd.concat([data.size(), data['VTmand [mL/kg]'].median(), data['VTmand [mL/kg]'].apply(iqr)], axis=1)
VT_mand = np.round(VT_mand, 1)
VT_mand.columns = ['number_of_minutes', 'median', 'IQR']

VT_spon = pd.concat([data.size(), data['VTspon [mL/kg]'].median(), data['VTspon [mL/kg]'].apply(iqr)], axis=1)
VT_spon = np.round(VT_spon, 1)
VT_spon.columns = ['number_of_minutes', 'median', 'IQR']

VT_spon_mand = pd.concat([data.size(), data['VT_spon_mand'].median(), data['VT_spon_mand'].apply(iqr)], axis=1)
VT_spon_mand = np.round(VT_spon_mand, 1)
VT_spon_mand.columns = ['number_of_minutes', 'median', 'IQR']

VT = pd.concat([VT_mand, VT_spon, VT_spon_mand], axis=1)
VT.columns = [['VT_mand', 'VT_mand', 'VT_mand', 
               'VT_spon', 'VT_spon', 'VT_spon', 
               'VT_spon_mand', 'VT_spon_mand', 'VT_spon_mand'], VT.columns]
VT

In [ ]:
np.round(VT.describe(), 2)

#### C. `Respiratory rate` of spontaneous and mandatory breaths

In [ ]:
%%time

dpi=200
filetype = 'pdf'

for patient in data_combined_selected_1min_mean:
    
    data = data_combined_selected_1min_mean[patient]
    
    print(f'Working on {patient}')
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)
    
    data['RRspon [1/min]'].plot(ax = ax, color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
    data['RRmand [1/min]'].plot(ax = ax, color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
            
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.set_ylim(-5, 110)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('1/min', size = 12, color = 'black')
    ax.set_title(patient,  size = 12, color = 'black')
    ax.legend(['RRspon', 'RRmand', ], ncol = 3, fontsize = 12, loc = 'best', markerscale=6)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
    fig.savefig(os.path.join(DATA_DUMP, f'{patient}_RR_mand_spon_1min_mean.{filetype}'), 
                dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)
    if patient != 'LVD002':
        plt.close()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 200
filetype = 'pdf'

data = data_combined_selected_1min_mean_all.dropna(how='any', subset = ['RRmand [1/min]', 'RRspon [1/min]'])
corr = stats.pearsonr(data['RRmand [1/min]'], data['RRspon [1/min]'])

ax.scatter(data['RRmand [1/min]'], data['RRspon [1/min]'], 
    marker = 'o', s=10, facecolor = 'black', edgecolor = 'blue', alpha = 0.005)
ax.text(5, 110, 'r: %.2f' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, 80)
ax.set_ylim(-10, 130)
ax.set_xlabel('RRmand [1/min]', size = 14)
ax.set_ylabel('RRspon [1/min]', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig(os.path.join(DIR_WRITE, f'scatterplot_RRmand_RRspon.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi=200
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (16,4))

data = data_combined_selected_1min_mean_all
data = list(data['RR_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

ax.boxplot(data_for_graph, whis = [5,95], patch_artist = True, showfliers = False,
    showmeans = True, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_xticklabels(xticklabels)
ax.set_ylabel('RRspon / RRmand')
    
fig.savefig(os.path.join(DIR_WRITE, f'RR_spon_mand_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

In [ ]:
data = data_combined_selected_1min_mean_all.dropna(how='any', subset=['RRmand [1/min]', 'RRspon [1/min]']).groupby(level=1)

RR_mand = pd.concat([data.size(), data['RRmand [1/min]'].median(), data['RRmand [1/min]'].apply(iqr)], axis=1)
RR_mand = np.round(RR_mand, 1)
RR_mand.columns = ['number_of_minutes', 'median', 'IQR']

RR_spon = pd.concat([data.size(), data['RRspon [1/min]'].median(), data['RRspon [1/min]'].apply(iqr)], axis=1)
RR_spon = np.round(RR_spon, 1)
RR_spon.columns = ['number_of_minutes', 'median', 'IQR']

RR_spon_mand = pd.concat([data.size(), data['RR_spon_mand'].median(), data['RR_spon_mand'].apply(iqr)], axis=1)
RR_spon_mand = np.round(RR_spon_mand, 1)
RR_spon_mand.columns = ['number_of_minutes', 'median', 'IQR']

RR = pd.concat([RR_mand, RR_spon, RR_spon_mand], axis=1)
RR.columns = [['RR_mand', 'RR_mand', 'RR_mand', 
               'RR_spon', 'RR_spon', 'RR_spon', 
               'RR_spon_mand', 'RR_spon_mand', 'RR_spon_mand'], RR.columns]
RR

In [ ]:
np.round(RR.describe(), 2)

#### D. `Expired minute ventilation` of the spontaneous and mandatory breaths

In [ ]:
%%time

dpi=200
filetype = 'pdf'

for patient in data_combined_selected_1min_mean:
    data = data_combined_selected_1min_mean[patient]
    print(f'Working on {patient}')
    
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)
    
    data['MVespon [L/min/kg]'].plot(ax = ax, color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
    data['MVemand [L/min/kg]'].plot(ax = ax, color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
            
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.set_ylim(-0.1, 1)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('L/min/kg', size = 12, color = 'black')
    ax.set_title(patient,  size = 12, color = 'black')
    ax.legend(['MVespon', 'MVemand', ], ncol = 3, fontsize = 12, loc = 'best', markerscale=6)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
    fig.savefig(os.path.join(DATA_DUMP, f'{patient}_MVemand_espon_1min_mean.{filetype}'), 
                dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)
    if patient != 'LVD002':
        plt.close()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 200
filetype = 'pdf'

data = data_combined_selected_1min_mean_all.dropna(how='any', subset = ['MVemand [L/min/kg]', 'MVespon [L/min/kg]'])
corr = stats.pearsonr(data['MVemand [L/min/kg]'], data['MVespon [L/min/kg]'])

ax.scatter(data['MVemand [L/min/kg]'], data['MVespon [L/min/kg]'], 
           marker = 'o', s=10, facecolor = 'black', edgecolor = 'blue', alpha = 0.005)
ax.text(0.1, 0.5, 'r: %.2f' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, 0.6)
ax.set_ylim(-0.1, 0.6)
ax.set_xlabel('MVemand [L/min/kg]', size = 14)
ax.set_ylabel('MVespon [L/min/kg]', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig(os.path.join(DIR_WRITE, f'scatterplot_MVemand_MVespon.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi=200
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (16,4))

data = data_combined_selected_1min_mean_all
data = list(data['MVe_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

ax.boxplot(data_for_graph, whis = [5,95], patch_artist = True, showfliers = False,
    showmeans = True, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_xticklabels(xticklabels)
ax.set_ylabel('MVespon / MVemand')
    
fig.savefig(os.path.join(DIR_WRITE, f'MVe_spon_mand_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

In [ ]:
%%time

data = data_combined_selected_1min_mean_all.dropna(how='any', 
            subset=['MVemand [L/min/kg]', 'MVespon [L/min/kg]', 'MVe_spon_mand']).groupby(level=1)

MVe_mand = pd.concat([data.size(), data['MVemand [L/min/kg]'].median(), data['MVemand [L/min/kg]'].apply(iqr)], axis=1)
MVe_mand = np.round(MVe_mand, 2)
MVe_mand.columns = ['number_of_minutes', 'median', 'IQR']


MVe_spon = pd.concat([data.size(), data['MVespon [L/min/kg]'].median(), data['MVespon [L/min/kg]'].apply(iqr)], axis=1)
MVe_spon = np.round(MVe_spon, 2)
MVe_spon.columns = ['number_of_minutes', 'median', 'IQR']

MVe_spon_mand = pd.concat([data.size(), data['MVe_spon_mand'].median(), 
                           data['MVe_spon_mand'].apply(iqr)], axis=1)
MVe_spon_mand = np.round(MVe_spon_mand, 2)
MVe_spon_mand.columns = ['number_of_minutes', 'median', 'IQR']

MVe = pd.concat([MVe_mand, MVe_spon, MVe_spon_mand], axis=1)
MVe.columns = [['MVe_mand', 'MVe_mand', 'MVe_mand', 
                'MVe_spon', 'MVe_spon', 'MVe_spon', 
                'MVe_spon_mand', 'MVe_spon_mand', 'MVe_spon_mand'], MVe.columns]
MVe

In [ ]:
np.round(MVe.describe(), 2)

#### E. `Inspiratory times` of the mandatory and spontaneous breaths

In [ ]:
%%time

dpi=200
filetype = 'pdf'

for patient in data_combined_selected_1min_mean:
    
    data = data_combined_selected_1min_mean[patient]
    print(f'Working on {patient}')
    
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)
    
    data['Tispon [s]'].plot(ax = ax, color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
    data['Ti_set [s]'].plot(ax = ax, color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
            
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.set_ylim(-0.1, 1)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('seconds', size = 12, color = 'black')
    ax.set_title(patient,  size = 12, color = 'black')
    ax.legend(['Tispon', 'Timand', ], ncol = 3, fontsize = 12, loc = 'best', markerscale=6)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
    fig.savefig(os.path.join(DATA_DUMP, f'{patient}_Timand_spon_1min_mean.{filetype}'), 
                dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)
    if patient != 'LVD002':
        plt.close()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 200
filetype = 'pdf'

data = data_combined_selected_1min_mean_all.dropna(subset = ['Ti_set [s]', 'Tispon [s]'])
corr = stats.pearsonr(data['Ti_set [s]'], data['Tispon [s]'])

ax.scatter(data['Ti_set [s]'], data['Tispon [s]'], 
           marker = 'o', s=10, facecolor = 'black', edgecolor = 'blue', alpha = 0.005)
ax.text(0.1, 0.5, 'r: %.2f' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, 0.6)
ax.set_ylim(-0.05, 0.6)
ax.set_xlabel('Ti_set [s]', size = 14)
ax.set_ylabel('Tispon [s]', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig(os.path.join(DIR_WRITE, f'scatterplot_Timand_Tispon.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
data = data_combined_selected_1min_mean_all.dropna(subset=['Ti_set [s]', 'Tispon [s]']).groupby(level=1)

Ti_mand = pd.concat([data.size(), data['Ti_set [s]'].median(), data['Ti_set [s]'].apply(iqr)], axis=1)
Ti_mand = np.round(Ti_mand, 2)
Ti_mand.columns = ['number_of_minutes', 'median', 'IQR']

Ti_spon = pd.concat([data.size(), data['Tispon [s]'].median(), data['Tispon [s]'].apply(iqr)], axis=1)
Ti_spon = np.round(Ti_spon, 2)
Ti_spon.columns = ['number_of_minutes', 'median', 'IQR']

Ti_spon_mand = pd.concat([data.size(), data['Ti_spon_mand'].median(), data['Ti_spon_mand'].apply(iqr)], axis=1)
Ti_spon_mand = np.round(Ti_spon_mand, 2)
Ti_spon_mand.columns = ['number_of_minutes', 'median', 'IQR']

Ti = pd.concat([Ti_mand, Ti_spon, Ti_spon_mand], axis=1)
Ti.columns = [['Ti_mand', 'Ti_mand', 'Ti_mand', 
               'Ti_spon', 'Ti_spon', 'Ti_spon',
               'Ti_spon_mand', 'Ti_spon_mand', 'Ti_spon_mand'], Ti.columns]
Ti

In [ ]:
np.round(Ti.describe(), 2)

In [ ]:
stats.mannwhitneyu(Ti_mand['median'], Ti_spon['median'])

#### Combine statistics in a single DataFrame and export

In [ ]:
statistics_1_min_mean = pd.concat([PIP, VT, RR, MVe, Ti], axis=1)
statistics_1_min_mean

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'stats_1_min_mean.xlsx'))
statistics_1_min_mean.to_excel(writer, 'statistics_1_min_mean')
writer.save()

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'group_stats_1_min_mean.xlsx'))
PIP.describe().to_excel(writer, 'PIP')
VT.describe().to_excel(writer, 'VT')
RR.describe().to_excel(writer, 'RR')
MVe.describe().to_excel(writer, 'MVe')
Ti.describe().to_excel(writer, 'Ti')
writer.save()

### 15. Visualise and analyse relationship between PIP and VT, of mandatory ventilator inflations and spontaneous breaths. Only consider minutes when the mean PIPspon/PIPmand was between 0.6 - 0.72 and analyse the relationship between spontaneous and mandatory tidal volumes in the subset

#### A. `peak inspiratory pressure` of mandatory and spontaneous breaths

In [ ]:
%%time

data_1min_mean_inrange = {}

for patient in data_combined_selected_1min_mean:
    data = data_combined_selected_1min_mean[patient]
    data = data[((data['PIP_spon [mbar]'] / data['PIP [mbar]']) >= 0.6) &  
                ((data['PIP_spon [mbar]'] / data['PIP [mbar]']) <= 0.72)]
    data_1min_mean_inrange[patient] = data

In [ ]:
data_1min_mean_inrange_all = pd.concat(data_1min_mean_inrange).swaplevel()
data_1min_mean_inrange_all.shape

In [ ]:
%%time

dpi=200
filetype = 'pdf'

for patient in data_1min_mean_inrange:
    data = data_1min_mean_inrange[patient]
    
    print(f'Working on {patient}')
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)
     
    data['PIP [mbar]'].plot(ax = ax, color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
    data['PIP_spon [mbar]'].plot(ax = ax, color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
            
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.set_ylim(-2, 35)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mbar', size = 12, color = 'black')
    ax.set_title(patient,  size = 12, color = 'black')
    ax.legend(['PIP', 'PIPspon',], ncol = 3, fontsize = 12, loc = 'best', markerscale=6)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
    fig.savefig(os.path.join(DATA_DUMP, f'{patient}_PIPmand_spon_1min_mean_inrange.{filetype}'), 
        dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)
    if patient != 'LVD002':
        plt.close()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 200
filetype = 'pdf'

data = data_1min_mean_inrange_all.dropna(subset = ['PIP [mbar]', 'PIP_spon [mbar]'])
corr = stats.pearsonr(data['PIP [mbar]'], data['PIP_spon [mbar]'])

ax.scatter(data['PIP [mbar]'], data['PIP_spon [mbar]'], 
        marker = 'o', s=10, facecolor = 'black', edgecolor = 'blue', alpha = 0.05)
ax.text(2, 18, 'r: %.2f' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, 40)
ax.set_ylim(-1, 20)
ax.set_xlabel('PIPmand [mbar]', size = 14)
ax.set_ylabel('PIPspon [mbar]', size = 14)
#ax.yaxis.set_major_formatter(ScalarFormatter())
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig(os.path.join(DIR_WRITE, f'corr_PIPmand_PIPspon_inrange.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
dpi=200
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (16,4))

data = list(data_1min_mean_inrange_all['PIP_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

ax.boxplot(data_for_graph, whis = [5,95], patch_artist = True, showfliers = False,
    showmeans = True, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 1.5)
ax.hlines(0.66, *ax.get_xlim(), color='red', linestyle='--')
ax.set_xticklabels(xticklabels)
ax.set_ylabel('PIPspon / PIPmand')
    
fig.savefig(os.path.join(DIR_WRITE, f'PIP_spon_mand_inrange_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

In [ ]:
data = data_1min_mean_inrange_all.dropna(how='any', subset=['PIP [mbar]', 'PIP_spon [mbar]']).groupby(level=1)

PIP_mand_inrange = pd.concat([data.size(), data['PIP [mbar]'].median(), data['PIP [mbar]'].apply(iqr)], axis=1)
PIP_mand_inrange = np.round(PIP_mand_inrange, 2)
PIP_mand_inrange.columns = ['number_of_minutes', 'median', 'IQR']

PIP_spon_inrange = pd.concat([data.size(), data['PIP_spon [mbar]'].median(), 
                              data['PIP_spon [mbar]'].apply(iqr)], axis=1)
PIP_spon_inrange = np.round(PIP_spon_inrange, 2)
PIP_spon_inrange.columns = ['number_of_minutes', 'median', 'IQR']

PIP_spon_mand_inrange = pd.concat([data.size(), data['PIP_spon_mand'].median(), 
                                   data['PIP_spon_mand'].apply(iqr)], axis=1)
PIP_spon_mand_inrange = np.round(PIP_spon_mand_inrange, 2)
PIP_spon_mand_inrange.columns = ['number_of_minutes', 'median', 'IQR']

PIP_inrange = pd.concat([PIP_mand_inrange, PIP_spon_inrange, PIP_spon_mand_inrange], axis=1)
PIP_inrange.columns = [['PIP_mand', 'PIP_mand', 'PIP_mand', 
                        'PIP_spon', 'PIP_spon', 'PIP_spon',
                        'PIP_spon_mand', 'PIP_spon_mand', 'PIP_spon_mand'], PIP_inrange.columns]

In [ ]:
PIP_inrange

#### B. `Leak-compensated expired tidal volume` of the spontaneous and mandatory breaths

In [ ]:
%%time

dpi=200
filetype = 'pdf'

for patient in data_1min_mean_inrange:
    data = data_1min_mean_inrange[patient]
    print(f'Working on {patient}')
    fig = plt.figure()
    fig.set_size_inches(12, 4)
    fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
    ax = fig.add_subplot(1, 1, 1)
    
    data['VTspon [mL/kg]'].plot(ax = ax, color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
    data['VTmand [mL/kg]'].plot(ax = ax, color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
    data['VT_set [mL/kg]'].plot(ax = ax, color = 'black', linewidth=2, linestyle='dashed', x_compat = True)
            
    ax.xaxis.set_major_formatter(majorFmt)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
    ax.set_ylim(-2, 25)
    ax.set_xlabel('Time (hours)', size = 12, color = 'black')
    ax.set_ylabel('mL/kg', size = 12, color = 'black')
    ax.set_title(patient,  size = 12, color = 'black')
    ax.legend(['VTspon', 'VTmand', 'VTset'], ncol = 3, fontsize = 12, loc = 'best', markerscale=6)
    ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
    fig.savefig(os.path.join(DATA_DUMP, f'{patient}_VTmand_spon_1min_mean_inrange.{filetype}'), 
                dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)
    if patient != 'LVD002':
        plt.close()

In [ ]:
fig, ax = plt.subplots(figsize = (6, 6))
dpi = 200
filetype = 'pdf'

data = data_1min_mean_inrange_all.dropna(subset = ['VTmand [mL/kg]', 'VTspon [mL/kg]'])
corr = stats.pearsonr(data['VTmand [mL/kg]'], data['VTspon [mL/kg]'])

ax.scatter(data['VTmand [mL/kg]'], data['VTspon [mL/kg]'], 
        marker = 'o', s=10, facecolor = 'black', edgecolor = 'blue', alpha = 0.05)
ax.text(2, 18, 'r: %.2f' % corr[0], fontsize = 14)
ax.grid(True)
ax.set_xlim(0, 20)
ax.set_ylim(-1, 20)
ax.set_xlabel('VTmand [mL/kg]', size = 14)
ax.set_ylabel('VTspon [mL/kg]', size = 14)
#ax.yaxis.set_major_formatter(ScalarFormatter())
ax.tick_params(axis='both', which='major', labelsize=14)

fig.savefig(os.path.join(DIR_WRITE, f'corr_VTmand_VTspon_inrange.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1);

In [ ]:
data = data_1min_mean_inrange_all.dropna(how='any', subset=['VTmand [mL/kg]', 'VTspon [mL/kg]']).groupby(level=1)

VT_mand_inrange = pd.concat([data.size(), data['VTmand [mL/kg]'].median(), 
                             data['VTmand [mL/kg]'].apply(iqr)], axis=1)
VT_mand_inrange = np.round(VT_mand_inrange, 2)
VT_mand_inrange.columns = ['number_of_minutes', 'median', 'IQR']

VT_spon_inrange = pd.concat([data.size(), data['VTspon [mL/kg]'].median(), 
                             data['VTspon [mL/kg]'].apply(iqr)], axis=1)
VT_spon_inrange = np.round(VT_spon_inrange, 2)
VT_spon_inrange.columns = ['number_of_minutes', 'median', 'IQR']

VT_spon_mand_inrange = pd.concat([data.size(), data['VT_spon_mand'].median(), 
                             data['VT_spon_mand'].apply(iqr)], axis=1)
VT_spon_mand_inrange = np.round(VT_spon_mand_inrange, 2)
VT_spon_mand_inrange.columns = ['number_of_minutes', 'median', 'IQR']

VT_inrange = pd.concat([VT_mand_inrange, VT_spon_inrange, VT_spon_mand_inrange], axis=1)
VT_inrange.columns = [['VT_mand', 'VT_mand', 'VT_mand', 
                        'VT_spon', 'VT_spon', 'VT_spon',
                        'VT_spon_mand', 'VT_spon_mand', 'VT_spon_mand'], VT_inrange.columns]

In [ ]:
VT_inrange

#### Combine statistics in a single DataFrame and export

In [ ]:
statistics_1_min_mean_inrange = pd.concat([PIP_inrange, VT_inrange], axis=1)
statistics_1_min_mean_inrange

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'stats_1_min_mean_inrange.xlsx'))
statistics_1_min_mean_inrange.to_excel(writer, 'stats_inrange')
PIP_inrange.describe().to_excel(writer, 'group_stats_PIP_inrange')
VT_inrange.describe().to_excel(writer, 'group_stats_VT_inrange')
writer.save()

#### Inferential statistics to compare 

In [ ]:
VT['spon_mand_IQR_range'] = VT['VT_spon_mand']['IQR'].apply(lambda x : x[1] - x[0])
VT_inrange['spon_mand_IQR_range'] = VT_inrange['VT_spon_mand']['IQR'].apply(lambda x : x[1] - x[0])

In [ ]:
VT

In [ ]:
VT_inrange

In [ ]:
VT.describe()['VT_spon']['median']

In [ ]:
VT_inrange.describe()['VT_spon']['median']

In [ ]:
stats.mannwhitneyu(VT['VT_spon']['median'] , VT_inrange['VT_spon']['median'])

In [ ]:
VT.describe()['VT_spon_mand']['median']

In [ ]:
VT_inrange.describe()['VT_spon_mand']['median']

In [ ]:
stats.mannwhitneyu(VT['VT_spon_mand']['median'] , VT_inrange['VT_spon_mand']['median'])

### 16. Examine ventilation parameters and tcCO2 for various ranges PIP_mand_spon 

#### Over 10 minute periods

Ventilator parameters are averages over the 10-minute period

In [ ]:
columns_to_look = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand',
                   'RRspon [1/min]', 'RRmand [1/min]', 'RR_spon_mand',
                   'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand']

bins = [0, 0.65, 99]
# bins = [0, 0.65, 0.99, 99]
labels = ['<0.66', '>=0.66']
#labels = ['<0.66', '0.66-0.99', '>=1']

cats_PIP_spon_mand_10min = {}

for patient in data_combined_selected_10min_mean:
    
    PIP_spon_mand_bins = pd.cut(data_combined_selected_10min_mean[patient]['PIP_spon_mand'], 
        bins=bins, right=True, labels=labels)
    cats_PIP_spon_mand_10min[patient] = data_combined_selected_10min_mean[patient].groupby(PIP_spon_mand_bins)
    cats_PIP_spon_mand_10min[patient] = pd.concat([cats_PIP_spon_mand_10min[patient][columns_to_look].median(),
                                            cats_PIP_spon_mand_10min[patient][columns_to_look].size()], axis=1)
    cats_PIP_spon_mand_10min[patient] = cats_PIP_spon_mand_10min[patient].rename({0: 'number_of_episodes'}, axis=1)
                                      
cats_PIP_spon_mand_10min_all = pd.concat(cats_PIP_spon_mand_10min).unstack()
cats_PIP_spon_mand_10min_all.head()

In [ ]:
pars_to_test = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand', 'RRspon [1/min]', 'RRmand [1/min]', 
                'RR_spon_mand', 'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand']

stat_results = {}

for par in pars_to_test:
    data = cats_PIP_spon_mand_10min_all[par].dropna()
    stat_results[par] = {'median_<0.66' : data['<0.66'].median(), 'IQR_<0.66' : iqr(data['<0.66']),
                         'median_>=0.66' : data['>=0.66'].median(), 'IQR_>=0.66' : iqr(data['>=0.66']),
                         'p_value' : stats.wilcoxon(data['<0.66'], data['>=0.66'])[1]}

cats_PIP_spon_mand_10min_all_stats = DataFrame(stat_results).T
cats_PIP_spon_mand_10min_all_stats

##### Test only those 10-minute periods when tcCO2 data are available

In [ ]:
columns_to_look = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand',
                   'RRspon [1/min]', 'RRmand [1/min]', 'RR_spon_mand',
                   'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand', 'tcCO2']

bins = [0, 0.65, 99]
# bins = [0, 0.65, 0.99, 99]
labels = ['<0.66', '>=0.66']
#labels = ['<0.66', '0.66-0.99', '>=1']

cats_PIP_spon_mand_10min_with_tcCO2 = {}

for patient in data_combined_selected_10min_mean_with_CO2:
    
    PIP_spon_mand_bins_with_tcCO2 = pd.cut(data_combined_selected_10min_mean_with_CO2[patient]['PIP_spon_mand'], 
        bins=bins, right=True, labels=labels)
    cats_PIP_spon_mand_10min_with_tcCO2[patient] = data_combined_selected_10min_mean_with_CO2[patient].groupby(PIP_spon_mand_bins_with_tcCO2)
    cats_PIP_spon_mand_10min_with_tcCO2[patient] = pd.concat([cats_PIP_spon_mand_10min_with_tcCO2[patient][columns_to_look].median(),
                                                        cats_PIP_spon_mand_10min_with_tcCO2[patient][columns_to_look].size()], axis=1)
    cats_PIP_spon_mand_10min_with_tcCO2[patient] = cats_PIP_spon_mand_10min_with_tcCO2[patient].rename({0: 'number_of_episodes'}, axis=1)
                                      

cats_PIP_spon_mand_10min_with_tcCO2_all = pd.concat(cats_PIP_spon_mand_10min_with_tcCO2).unstack()
cats_PIP_spon_mand_10min_with_tcCO2_all

In [ ]:
pars_to_test = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand', 'RRspon [1/min]', 'RRmand [1/min]', 
                'RR_spon_mand', 'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand',  
                'tcCO2']

stat_results = {}

for par in pars_to_test:
    data = cats_PIP_spon_mand_10min_with_tcCO2_all[par].dropna()
    stat_results[par] = {'median_<0.66' : data['<0.66'].median(), 'IQR_<0.66' : iqr(data['<0.66']),
                         'median_>=0.66' : data['>=0.66'].median(), 'IQR_>=0.66' : iqr(data['>=0.66']),
                         'p_value' : stats.wilcoxon(data['<0.66'], data['>=0.66'])[1]}

cats_PIP_spon_mand_10min_with_tcCO2_all_stats = DataFrame(stat_results).T
cats_PIP_spon_mand_10min_with_tcCO2_all_stats

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'cats_PIP_spon_mand_10min.xlsx'))
cats_PIP_spon_mand_10min_all.to_excel(writer, 'all')
cats_PIP_spon_mand_10min_all_stats.to_excel(writer, 'stats')
cats_PIP_spon_mand_10min_with_tcCO2_all.to_excel(writer, 'with_CO2_all')
cats_PIP_spon_mand_10min_with_tcCO2_all_stats.to_excel(writer, 'stats_with_CO2')
writer.save()

#### Over 1-hour periods

Ventilator parameters are averages over the 1 hour, tcCO2 is average over the last 10 minutes of the hour

In [ ]:
columns_to_look = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand',
                   'RRspon [1/min]', 'RRmand [1/min]', 'RR_spon_mand',
                   'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand']

bins = [0, 0.65, 99]
# bins = [0, 0.65, 0.99, 99]
labels = ['<0.66', '>=0.66']
#labels = ['<0.66', '0.66-0.99', '>=1']

cats_PIP_spon_mand_1hour = {}

for patient in data_combined_selected_1hour_mean:
    
    PIP_spon_mand_bins = pd.cut(data_combined_selected_1hour_mean[patient]['PIP_spon_mand'], 
        bins=bins, right=True, labels=labels)
    cats_PIP_spon_mand_1hour[patient] = data_combined_selected_1hour_mean[patient].groupby(PIP_spon_mand_bins)
    cats_PIP_spon_mand_1hour[patient] = pd.concat([cats_PIP_spon_mand_1hour[patient][columns_to_look].median(),
                                            cats_PIP_spon_mand_1hour[patient][columns_to_look].size()], axis=1)
    cats_PIP_spon_mand_1hour[patient] = cats_PIP_spon_mand_1hour[patient].rename({0: 'number_of_episodes'}, axis=1)
                                      

cats_PIP_spon_mand_1hour_all = pd.concat(cats_PIP_spon_mand_1hour).unstack()
cats_PIP_spon_mand_1hour_all.head()

In [ ]:
pars_to_test = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand', 'RRspon [1/min]', 'RRmand [1/min]', 
                'RR_spon_mand', 'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand']

stat_results = {}

for par in pars_to_test:
    data = cats_PIP_spon_mand_1hour_all[par].dropna()
    stat_results[par] = {'median_<0.66' : data['<0.66'].median(), 'IQR_<0.66' : iqr(data['<0.66']),
                         'median_>=0.66' : data['>=0.66'].median(), 'IQR_>=0.66' : iqr(data['>=0.66']),
                         'p_value' : stats.wilcoxon(data['<0.66'], data['>=0.66'])[1]}

cats_PIP_spon_mand_1hour_all_stats = DataFrame(stat_results).T
cats_PIP_spon_mand_1hour_all_stats

##### Test only those 1-hour periods when tcCO2 data are available

In [ ]:
columns_to_look = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand',
                   'RRspon [1/min]', 'RRmand [1/min]', 'RR_spon_mand',
                   'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand', 'tcCO2']

bins = [0, 0.65, 99]
# bins = [0, 0.65, 0.99, 99]
labels = ['<0.66', '>=0.66']
#labels = ['<0.66', '0.66-0.99', '>=1']

cats_PIP_spon_mand_1hour_with_tcCO2 = {}

for patient in data_combined_selected_1hour_mean_with_CO2:
    
    PIP_spon_mand_bins_with_tcCO2 = pd.cut(data_combined_selected_1hour_mean_with_CO2[patient]['PIP_spon_mand'], 
        bins=bins, right=True, labels=labels)
    cats_PIP_spon_mand_1hour_with_tcCO2[patient] = data_combined_selected_1hour_mean_with_CO2[patient].groupby(PIP_spon_mand_bins_with_tcCO2)
    cats_PIP_spon_mand_1hour_with_tcCO2[patient] = pd.concat([cats_PIP_spon_mand_1hour_with_tcCO2[patient][columns_to_look].median(),
                                            cats_PIP_spon_mand_1hour_with_tcCO2[patient][columns_to_look].size()], axis=1)
    cats_PIP_spon_mand_1hour_with_tcCO2[patient] = cats_PIP_spon_mand_1hour_with_tcCO2[patient].rename({0: 'number_of_episodes'}, axis=1)
                                      

cats_PIP_spon_mand_1hour_with_tcCO2_all = pd.concat(cats_PIP_spon_mand_1hour_with_tcCO2).unstack()
cats_PIP_spon_mand_1hour_with_tcCO2_all

In [ ]:
pars_to_test = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand', 'RRspon [1/min]', 'RRmand [1/min]', 
                'RR_spon_mand', 'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand', 'tcCO2']

stat_results = {}

for par in pars_to_test:
    data = cats_PIP_spon_mand_1hour_with_tcCO2_all[par].dropna()
    stat_results[par] = {'median_<0.66' : data['<0.66'].median(), 'IQR_<0.66' : iqr(data['<0.66']),
                         'median_>=0.66' : data['>=0.66'].median(), 'IQR_>=0.66' : iqr(data['>=0.66']),
                         'p_value' : stats.wilcoxon(data['<0.66'], data['>=0.66'])[1]}

cats_PIP_spon_mand_1hour_with_tcCO2_all_stats = DataFrame(stat_results).T
cats_PIP_spon_mand_1hour_with_tcCO2_all_stats

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'cats_PIP_spon_mand_1hour.xlsx'))
cats_PIP_spon_mand_1hour_all.to_excel(writer, 'all')
cats_PIP_spon_mand_1hour_all_stats.to_excel(writer, 'stats')
cats_PIP_spon_mand_1hour_with_tcCO2_all.to_excel(writer, 'with_CO2_all')
cats_PIP_spon_mand_1hour_with_tcCO2_all_stats.to_excel(writer, 'stats_with_CO2')
writer.save()

### 17.  Compare periods with small and large spontaneous tidal volumes

#### Over 10 minute periods

Ventilator parameters are averages over the 10-minute period

In [ ]:
# Discretize data into two groups of equal size
labels = ['VTspon_low', 'VTspon_high'] 

columns_to_look = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand',
                   'RRspon [1/min]', 'RRmand [1/min]', 'RR_spon_mand',
                   'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand']

cats_VTspon_10min = {}

for patient in data_combined_selected_10min_mean:
    
    VT_spon_bins = pd.qcut(data_combined_selected_10min_mean[patient]['VTspon [mL/kg]'], q=2, labels=labels)
    cats_VTspon_10min[patient] = \
        data_combined_selected_10min_mean[patient].groupby(VT_spon_bins)
    cats_VTspon_10min[patient] = pd.concat([cats_VTspon_10min[patient][columns_to_look].median(),
                                            cats_VTspon_10min[patient][columns_to_look].size()], axis=1)
    cats_VTspon_10min[patient] = cats_VTspon_10min[patient].rename({0: 'number_of_episodes'}, axis=1)
                                      

cats_VTspon_10min_all = pd.concat(cats_VTspon_10min).unstack()
cats_VTspon_10min_all

In [ ]:
pars_to_test = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand', 'RRspon [1/min]', 'RRmand [1/min]', 
                'RR_spon_mand', 'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand']

stat_results = {}

for par in pars_to_test:
    data = cats_VTspon_10min_all[par].dropna()
    stat_results[par] = {'low_median' : data['VTspon_low'].median(), 'low_IQR' : iqr(data['VTspon_low']),
                         'high_median' : data['VTspon_high'].median(), 'high_IQR' : iqr(data['VTspon_high']),
                         'p_value' : stats.wilcoxon(data['VTspon_low'], data['VTspon_high'])[1]}

cats_VTspon_10min_all_stats = DataFrame(stat_results).T
cats_VTspon_10min_all_stats

#### Only those 10-minute periods when transcutaneous data is available together with transcutaneous data

Consider the mean tcCO2 over the 10-minute period

In [ ]:
# Discretize data into two groups of equal size
labels = ['VTspon_low', 'VTspon_high'] 

columns_to_look = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand',
                   'RRspon [1/min]', 'RRmand [1/min]', 'RR_spon_mand',
                   'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand',
                    'tcCO2',]

cats_VTspon_10min_with_tcCO2 = {}

for patient in data_combined_selected_10min_mean_with_CO2:
    
    VT_spon_bins = pd.qcut(data_combined_selected_10min_mean_with_CO2[patient]['VTspon [mL/kg]'], 
                           q=2, labels=labels)
    cats_VTspon_10min_with_tcCO2[patient] = \
        data_combined_selected_10min_mean_with_CO2[patient].groupby(VT_spon_bins)
    cats_VTspon_10min_with_tcCO2[patient] = pd.concat([cats_VTspon_10min_with_tcCO2[patient][columns_to_look].median(),
                                            cats_VTspon_10min_with_tcCO2[patient][columns_to_look].size()], axis=1)
    cats_VTspon_10min_with_tcCO2[patient] = cats_VTspon_10min_with_tcCO2[patient].rename({0: 'number_of_episodes'}, axis=1)
                                      
cats_VTspon_10min_with_tcCO2_all = pd.concat(cats_VTspon_10min_with_tcCO2).unstack()
cats_VTspon_10min_with_tcCO2_all

In [ ]:
pars_to_test = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand', 'RRspon [1/min]', 'RRmand [1/min]', 
                'RR_spon_mand', 'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand', 'tcCO2',]

stat_results = {}

for par in pars_to_test:
    data = cats_VTspon_10min_with_tcCO2_all[par].dropna()
    stat_results[par] = {'low_median' : data['VTspon_low'].median(), 'low_IQR' : iqr(data['VTspon_low']),
                         'high_median' : data['VTspon_high'].median(), 'high_IQR' : iqr(data['VTspon_high']),
                         'p_value' : stats.wilcoxon(data['VTspon_low'], data['VTspon_high'])[1]}

cats_VTspon_10min_with_tcCO2_all_stats = DataFrame(stat_results).T
cats_VTspon_10min_with_tcCO2_all_stats

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'cats_VTspon_10min.xlsx'))
cats_VTspon_10min_all.to_excel(writer, 'all')
cats_VTspon_10min_all_stats.to_excel(writer, 'stats')
cats_VTspon_10min_with_tcCO2_all.to_excel(writer, 'with_CO2_all')
cats_VTspon_10min_with_tcCO2_all_stats.to_excel(writer, 'stats_with_CO2')
writer.save()

#### Over 1-hour periods

Ventilator parameters are averages over the 1 hour, tcCO2 is average over the last 10 minutes of the hour

In [ ]:
# Discretize data into two groups of equal size
labels = ['VTspon_low', 'VTspon_high'] 

columns_to_look = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand',
                   'RRspon [1/min]', 'RRmand [1/min]', 'RR_spon_mand',
                   'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand']

cats_VTspon_1hour = {}

for patient in data_combined_selected_1hour_mean:
    
    VT_spon_bins = pd.qcut(data_combined_selected_1hour_mean[patient]['VTspon [mL/kg]'], q=2, labels=labels)
    cats_VTspon_1hour[patient] = \
        data_combined_selected_1hour_mean[patient].groupby(VT_spon_bins)
    cats_VTspon_1hour[patient] = pd.concat([cats_VTspon_1hour[patient][columns_to_look].median(),
                                            cats_VTspon_1hour[patient][columns_to_look].size()], axis=1)
    cats_VTspon_1hour[patient] = cats_VTspon_1hour[patient].rename({0: 'number_of_episodes'}, axis=1)
                                      

cats_VTspon_1hour_all = pd.concat(cats_VTspon_1hour).unstack()
cats_VTspon_1hour_all

In [ ]:
pars_to_test = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand', 'RRspon [1/min]', 'RRmand [1/min]', 
                'RR_spon_mand', 'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand']

stat_results = {}

for par in pars_to_test:
    data = cats_VTspon_1hour_all[par].dropna()
    stat_results[par] = {'low_median' : data['VTspon_low'].median(), 'low_IQR' : iqr(data['VTspon_low']),
                         'high_median' : data['VTspon_high'].median(), 'high_IQR' : iqr(data['VTspon_high']),
                         'p_value' : stats.wilcoxon(data['VTspon_low'], data['VTspon_high'])[1]}

cats_VTspon_1hour_all_stats = DataFrame(stat_results).T
cats_VTspon_1hour_all_stats

#### Only those 10-minute periods when transcutaneous data is available together with transcutaneous data

Consider the mean tcCO2 over the 10-minute period

In [ ]:
# Discretize data into two groups of equal size
labels = ['VTspon_low', 'VTspon_high'] 

columns_to_look = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand',
                   'RRspon [1/min]', 'RRmand [1/min]', 'RR_spon_mand',
                   'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand',
                    'tcCO2',]

cats_VTspon_1hour_with_tcCO2 = {}

for patient in data_combined_selected_1hour_mean_with_CO2:
    
    VT_spon_bins = pd.qcut(data_combined_selected_1hour_mean_with_CO2[patient]['VTspon [mL/kg]'], 
                           q=2, labels=labels)
    cats_VTspon_1hour_with_tcCO2[patient] = \
        data_combined_selected_1hour_mean_with_CO2[patient].groupby(VT_spon_bins)
    cats_VTspon_1hour_with_tcCO2[patient] = pd.concat([cats_VTspon_1hour_with_tcCO2[patient][columns_to_look].median(),
                                            cats_VTspon_1hour_with_tcCO2[patient][columns_to_look].size()], axis=1)
    cats_VTspon_1hour_with_tcCO2[patient] = cats_VTspon_1hour_with_tcCO2[patient].rename({0: 'number_of_episodes'}, axis=1)
                                      
cats_VTspon_1hour_with_tcCO2_all = pd.concat(cats_VTspon_1hour_with_tcCO2).unstack()
cats_VTspon_1hour_with_tcCO2_all

In [ ]:
pars_to_test = ['VTspon [mL/kg]', 'VTmand [mL/kg]', 'VT_spon_mand', 'RRspon [1/min]', 'RRmand [1/min]', 
                'RR_spon_mand', 'MVespon [L/min/kg]', 'MVemand [L/min/kg]', 'MVe_spon_mand', 'tcCO2',]

stat_results = {}

for par in pars_to_test:
    data = cats_VTspon_1hour_with_tcCO2_all[par].dropna()
    stat_results[par] = {'low_median' : data['VTspon_low'].median(), 'low_IQR' : iqr(data['VTspon_low']),
                         'high_median' : data['VTspon_high'].median(), 'high_IQR' : iqr(data['VTspon_high']),
                         'p_value' : stats.wilcoxon(data['VTspon_low'], data['VTspon_high'])[1]}

cats_VTspon_1hour_with_tcCO2_all_stats = DataFrame(stat_results).T
cats_VTspon_1hour_with_tcCO2_all_stats

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'cats_VTspon_1hour.xlsx'))
cats_VTspon_1hour_all.to_excel(writer, 'all')
cats_VTspon_1hour_all_stats.to_excel(writer, 'stats')
cats_VTspon_1hour_with_tcCO2_all.to_excel(writer, 'with_CO2_all')
cats_VTspon_1hour_with_tcCO2_all_stats.to_excel(writer, 'stats_with_CO2')

writer.save()

### 18. Longitudinal analysis of the spon/mand ratios

In [ ]:
# Split each DataFrame containing the 1-minute averages to 4 equal parts ("quartiles")
data_combined_selected_1min_mean_split = {}

for patient in data_combined_selected_1min_mean:
    data_combined_selected_1min_mean_split[patient] = \
        np.array_split(data_combined_selected_1min_mean[patient], 4)

In [ ]:
# Calculate the median of the ratio of spon/mand for PIP, VT, RR, and MV
spon_mand_longitudinal = {}

parameters = ['PIP_spon_mand', 'VT_spon_mand', 'RR_spon_mand', 'MVe_spon_mand']
quartiles = ['1st_quartile', '2nd_quartile', '3rd_quartile', '4th_quartile']

for parameter in parameters:
    spon_mand_longitudinal[parameter] = []
    for patient in data_combined_selected_1min_mean_split:
        inner_lst = []
        for i, dta in enumerate(data_combined_selected_1min_mean_split[patient]):
            inner_lst.append(round(dta[parameter].median(), 3))
        spon_mand_longitudinal[parameter].append(inner_lst)
    spon_mand_longitudinal[parameter] = DataFrame(spon_mand_longitudinal[parameter], 
        columns = quartiles, index = data_combined_selected_1min_mean_split.keys())                                         

spon_mand_longitudinal_all = pd.concat(spon_mand_longitudinal, axis=1)
spon_mand_longitudinal_all

In [ ]:
f_test = {}

for parameter in parameters:
    stat_results = stats.friedmanchisquare(spon_mand_longitudinal[parameter]['1st_quartile'], 
                                           spon_mand_longitudinal[parameter]['2nd_quartile'],
                                           spon_mand_longitudinal[parameter]['3rd_quartile'],
                                           spon_mand_longitudinal[parameter]['4th_quartile'])
    f_test[parameter] = {'test': 'Friedman', 'statistic' : stat_results[0], 'pvalue': stat_results[1]}

f_test_all  = pd.DataFrame(f_test)
f_test_all

In [ ]:
wilc_test = {}

for parameter in parameters:
    df = spon_mand_longitudinal[parameter]
    stat_results = [stats.wilcoxon(df[i], df[j])[1] for i,j in itertools.combinations(df.columns,2)]
    wilc_test[parameter] = stat_results
    
wilc_test_all  = pd.DataFrame(wilc_test)
wilc_test_all.index = ["wilcoxon " + i+" vs "+j for i,j in itertools.combinations(df.columns,2)]
wilc_test_all

In [ ]:
writer =  pd.ExcelWriter(os.path.join(DIR_WRITE, 'longitudinal_analysis.xlsx'))
f_test_all.to_excel(writer, 'friedman_test')
wilc_test_all.to_excel(writer, 'wolcoxon_post_hoc_test')

writer.save()

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (5,3))

ax.boxplot(spon_mand_longitudinal['PIP_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 1.2)
ax.set_xticklabels(spon_mand_longitudinal['PIP_spon_mand'].columns)
ax.set_ylabel('PIP$_\mathrm{PS}$ / PIP$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, 'PIP_spon_mand_longit_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (5,3))

ax.boxplot(spon_mand_longitudinal['VT_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 1.4)
ax.set_xticklabels(spon_mand_longitudinal['VT_spon_mand'].columns)
ax.set_ylabel('VT$_\mathrm{PS}$ / VT$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'VT_spon_mand_longit_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (5,3))

ax.boxplot(spon_mand_longitudinal['RR_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 2.5)
ax.set_xticklabels(spon_mand_longitudinal['PIP_spon_mand'].columns)
ax.set_ylabel('RR$_\mathrm{PS}$ / RR$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'RR_spon_mand_longit_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (5,3))

ax.boxplot(spon_mand_longitudinal['MVe_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 2.5)
ax.set_xticklabels(spon_mand_longitudinal['MVe_spon_mand'].columns)
ax.set_ylabel('MV$_\mathrm{PS}$ / MV$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'MVe_spon_mand_longit_boxplot.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

### 19. Figures for the paper

#### Figure 1

##### Figure 1A

In [ ]:
dpi=300
filetype = 'pdf'
majorFmt = dates.DateFormatter('%d/%m')

patient = 'LVD002'
data = data_combined_selected_1min_mean

fig = plt.figure()
fig.set_size_inches(16, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)
    
data[patient]['PIP [mbar]'].plot(ax = ax, color = 'red',  marker='o', linestyle='', markersize=1, x_compat = True)
data[patient]['PIP_spon [mbar]'].plot(ax = ax, color = 'green', marker='o', linestyle='', markersize=1, x_compat = True)
        
ax.xaxis.set_major_formatter(majorFmt)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
ax.set_ylim(0, ax.get_ylim()[1])
ax.set_xlabel('Dates (day/month)', size = 12, color = 'black')
ax.set_ylabel('mbar', size = 12, color = 'black')
ax.set_title('',  size = 12, color = 'black')
ax.legend(['PIP$_\mathrm{VG}$', 'PIP$_\mathrm{PS}$',], ncol = 2, fontsize = 12, loc = 'best', markerscale=6)
#ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1A_col.{filetype}'), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

##### Figure 1B

In [ ]:
dpi=300
filetype = 'pdf'
majorFmt = dates.DateFormatter('%d/%m')

patient = 'LVD002'
data = data_combined_selected_1min_mean[patient]

fig = plt.figure()
fig.set_size_inches(12, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)
    
data['VTspon [mL/kg]'].plot(ax = ax, color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
data['VTmand [mL/kg]'].plot(ax = ax, color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
data['VT_set [mL/kg]'].plot(ax = ax, color = 'black', linewidth=2, linestyle='dashed', x_compat = True)
            
ax.xaxis.set_major_formatter(majorFmt)
plt.setp(ax.xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
ax.set_ylim(-2, 25)
ax.set_xlabel('Dates (day/month)', size = 12, color = 'black')
ax.set_ylabel('mL/kg', size = 12, color = 'black')
ax.set_title('',  size = 12, color = 'black')
ax.legend(['VT$_\mathrm{PS}$', 'VT$_\mathrm{VG}$', 'VT$_\mathrm{SET}$'], ncol = 3, fontsize = 12, loc = 'best', markerscale=6)
#ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
        
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1B_col.{filetype}'), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

##### Figure 1 combined (colored)

In [ ]:
dpi=300
filetype = 'eps'
majorFmt = dates.DateFormatter('%d/%m')

patient = 'LVD002'
data = data_combined_selected_1min_mean[patient]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize = [12, 8])
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, hspace=0.2, wspace=None)
  
# Figure 1A

data['PIP [mbar]'].plot(ax = axes[0], color = 'red',  marker='o', linestyle='', markersize=1, x_compat = True)
data['PIP_spon [mbar]'].plot(ax = axes[0],color = 'green', marker='o', linestyle='', markersize=1, x_compat = True)
        
axes[0].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].set_xlabel('', size = 12, color = 'black')
axes[0].set_ylabel('mbar', size = 12, color = 'black')
axes[0].set_title('',  size = 12, color = 'black')
axes[0].legend(['PIP$_\mathrm{VG}$', 'PIP$_\mathrm{PS}$',], ncol = 2, fontsize = 12, loc = 2, markerscale=6)
#axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[0].tick_params(bottom=False, labelbottom=False)
   
# Figure 1B

data['VTspon [mL/kg]'].plot(ax = axes[1], color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
data['VTmand [mL/kg]'].plot(ax = axes[1], color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
data['VT_set [mL/kg]'].plot(ax = axes[1], color = 'black', linewidth=2, linestyle='dashed', x_compat = True)
            
axes[1].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
axes[1].set_ylim(-2, 25)
axes[1].set_xlabel('Dates (day/month)', size = 12, color = 'black')
axes[1].set_ylabel('mL/kg', size = 12, color = 'black')
axes[1].set_title('',  size = 12, color = 'black')
axes[1].legend(['VT$_\mathrm{PS}$', 'VT$_\mathrm{VG}$', 'VT$_\mathrm{SET}$'], 
               ncol = 3, fontsize = 12, loc = 2, markerscale=6)
#axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.text(0.06, 0.90, 'A', fontsize = 16); fig.text(0.06, 0.54, 'B', fontsize = 16)
        
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1_col.{filetype}'), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

In [ ]:
dpi=300
filetype = 'jpg'
majorFmt = dates.DateFormatter('%d/%m')

patient = 'LVD002'
data = data_combined_selected_1min_mean[patient]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize = [12, 8])
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, hspace=0.2, wspace=None)
  
# Figure 1A

data['PIP [mbar]'].plot(ax = axes[0], color = 'red',  marker='o', linestyle='', markersize=1, x_compat = True)
data['PIP_spon [mbar]'].plot(ax = axes[0],color = 'green', marker='o', linestyle='', markersize=1, x_compat = True)
        
axes[0].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].set_xlabel('', size = 12, color = 'black')
axes[0].set_ylabel('mbar', size = 12, color = 'black')
axes[0].set_title('',  size = 12, color = 'black')
axes[0].legend(['PIPmand', 'PIPspon',], ncol = 2, fontsize = 12, loc = 2, markerscale=6)
#axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[0].tick_params(bottom=False, labelbottom=False)
   
# Figure 1B

data['VTspon [mL/kg]'].plot(ax = axes[1], color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
data['VTmand [mL/kg]'].plot(ax = axes[1], color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
data['VT_set [mL/kg]'].plot(ax = axes[1], color = 'black', linewidth=2, linestyle='dashed', x_compat = True)
            
axes[1].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
axes[1].set_ylim(-2, 25)
axes[1].set_xlabel('Dates (day/month)', size = 12, color = 'black')
axes[1].set_ylabel('mL/kg', size = 12, color = 'black')
axes[1].set_title('',  size = 12, color = 'black')
axes[1].legend(['VTspon', 'VTmand', 'VTset'], 
               ncol = 3, fontsize = 12, loc = 2, markerscale=6)
#axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.text(0.06, 0.90, 'A', fontsize = 16); fig.text(0.06, 0.54, 'B', fontsize = 16)
        
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1_alt_col.{filetype}'), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

##### Figure 1 combined (black and white)

In [ ]:
dpi=300
filetype = 'eps'
majorFmt = dates.DateFormatter('%d/%m')

patient = 'LVD002'
data = data_combined_selected_1min_mean[patient]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize = [12, 8])
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, hspace=0.1, wspace=None)
  
# Figure 1A

data['PIP [mbar]'].plot(ax = axes[0], color = 'gray',  marker='o', linestyle='', markersize=1, x_compat = True)
data['PIP_spon [mbar]'].plot(ax = axes[0],color = 'black', marker='o', linestyle='', markersize=1, x_compat = True)
        
axes[0].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
axes[0].set_ylim(0, axes[0].get_ylim()[1])
axes[0].set_xlabel('', size = 12, color = 'black')
axes[0].set_ylabel('mbar', size = 12, color = 'black')
axes[0].set_title('',  size = 12, color = 'black')
axes[0].legend(['PIP$_\mathrm{VG}$', 'PIP$_\mathrm{PS}$',], ncol = 2, fontsize = 12, loc = 2, markerscale=6)
#axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[0].tick_params(bottom=False, labelbottom=False)
   
# Figure 1B

data['VTspon [mL/kg]'].plot(ax = axes[1], color = 'black', linestyle='', marker='o', markersize=1, x_compat = True)
data['VTmand [mL/kg]'].plot(ax = axes[1], color = 'gray', linestyle='', marker='o', markersize=1, x_compat = True)
data['VT_set [mL/kg]'].plot(ax = axes[1], color = 'black', linewidth=2, linestyle='dashed', x_compat = True)
            
axes[1].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
axes[1].set_ylim(-2, 25)
axes[1].set_xlabel('Dates (day/month)', size = 12, color = 'black')
axes[1].set_ylabel('mL/kg', size = 12, color = 'black')
axes[1].set_title('',  size = 12, color = 'black')
axes[1].legend(['VT$_\mathrm{PS}$', 'VT$_\mathrm{VG}$', 'VT$_\mathrm{SET}$'], 
               ncol = 3, fontsize = 12, loc = 2, markerscale=6)
#axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.text(0.06, 0.90, 'A', fontsize = 16); fig.text(0.06, 0.54, 'B', fontsize = 16)
        
fig.savefig(os.path.join(DIR_WRITE, f'Figure_1.{filetype}'), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

#### Figure 2

##### Figure 2A

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (16,4))

data = list(data_combined_selected_1min_mean_all['PIP_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

ax.boxplot(data_for_graph, whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 2)
ax.hlines(0.66, *ax.get_xlim(), color='black', linestyle='--')
ax.set_xticklabels(xticklabels)
ax.set_ylabel('PIP$_\mathrm{PS}$ / PIP$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_2A.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

##### Figure 2B

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, ax = plt.subplots(figsize = (16,4))

data = data_combined_selected_1min_mean_all
data = list(data['VT_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

ax.boxplot(data_for_graph, whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 2)
ax.set_xticklabels(xticklabels)
ax.set_ylabel('VT$_\mathrm{PS}$ / VT$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_2B.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

##### Figure 2 combined

In [ ]:
dpi=300
filetype = 'eps'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, axes = plt.subplots(nrows=2, ncols=1, figsize = [12, 8])
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, hspace=0.1, wspace=None)

# Figure 2A

data = list(data_combined_selected_1min_mean_all['PIP_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

axes[0].boxplot(data_for_graph, whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

axes[0].set_ylim(0, 2)
axes[0].hlines(0.66, *axes[0].get_xlim(), color='black', linestyle='--', linewidth=1)
axes[0].set_xticklabels(xticklabels)
axes[0].set_ylabel('PIP$_\mathrm{PS}$ / PIP$_\mathrm{VG}$')
axes[0].tick_params(bottom=False, labelbottom=False)

# Figure 2B

data = list(data_combined_selected_1min_mean_all['VT_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

axes[1].boxplot(data_for_graph, whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

axes[1].set_ylim(0, 2)
axes[1].set_xticklabels(xticklabels)
axes[1].set_ylabel('VT$_\mathrm{PS}$ / VT$_\mathrm{VG}$')

fig.text(0.04, 0.90, 'A', fontsize = 16); fig.text(0.04, 0.48, 'B', fontsize = 16)
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_2.{filetype}'), dpi = dpi, format = filetype, 
    bbox_inches='tight', pad_inches=0.1)

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, axes = plt.subplots(nrows=2, ncols=1, figsize = [12, 8])
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, hspace=0.1, wspace=None)

# Figure 2A

data = list(data_combined_selected_1min_mean_all['PIP_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

axes[0].boxplot(data_for_graph, whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

axes[0].set_ylim(0, 2)
axes[0].hlines(0.66, *axes[0].get_xlim(), color='black', linestyle='--', linewidth=1)
axes[0].set_xticklabels(xticklabels)
axes[0].set_ylabel('PIPspon / PIPmand')
axes[0].tick_params(bottom=False, labelbottom=False)

# Figure 2B

data = list(data_combined_selected_1min_mean_all['VT_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

axes[1].boxplot(data_for_graph, whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

axes[1].set_ylim(0, 2)
axes[1].set_xticklabels(xticklabels)
axes[1].set_ylabel('VTspon / VTmand')

fig.text(0.04, 0.90, 'A', fontsize = 16); fig.text(0.04, 0.48, 'B', fontsize = 16)
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_2_alt.{filetype}'), dpi = dpi, format = filetype, 
    bbox_inches='tight', pad_inches=0.1)

#### Figure 3

##### Figure 3A

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

xticklabels = ['1$^\mathrm{st}$ quarter', '2$^\mathrm{nd}$ quarter', 
               '3$^\mathrm{rd}$ quarter', '4$^\mathrm{th}$ quarter']

fig, ax = plt.subplots(figsize = (5,3))

ax.boxplot(spon_mand_longitudinal['PIP_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 1.2)
ax.set_xticklabels(xticklabels)
ax.set_ylabel('PIP$_\mathrm{PS}$ / PIP$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_3A.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

##### Figure 3B

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

xticklabels = ['1$^\mathrm{st}$ quarter', '2$^\mathrm{nd}$ quarter', 
               '3$^\mathrm{rd}$ quarter', '4$^\mathrm{th}$ quarter']

fig, ax = plt.subplots(figsize = (5,3))

ax.boxplot(spon_mand_longitudinal['VT_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 1.4)
ax.set_xticklabels(xticklabels)
ax.set_ylabel('VT$_\mathrm{PS}$ / VT$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_3B.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

#### Figure 3C

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

xticklabels = ['1$^\mathrm{st}$ quarter', '2$^\mathrm{nd}$ quarter', 
               '3$^\mathrm{rd}$ quarter', '4$^\mathrm{th}$ quarter']

fig, ax = plt.subplots(figsize = (5,3))

ax.boxplot(spon_mand_longitudinal['RR_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 2.5)
ax.set_xticklabels(xticklabels)
ax.set_ylabel('RR$_\mathrm{PS}$ / RR$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_3C.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

#### Figure 3D

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

xticklabels = ['1$^\mathrm{st}$ quarter', '2$^\mathrm{nd}$ quarter', 
               '3$^\mathrm{rd}$ quarter', '4$^\mathrm{th}$ quarter']

fig, ax = plt.subplots(figsize = (5,3))

ax.boxplot(spon_mand_longitudinal['MVe_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax.set_ylim(0, 2.5)
ax.set_xticklabels(xticklabels)
ax.set_ylabel('MV$_\mathrm{PS}$ / MV$_\mathrm{VG}$')
    
fig.savefig(os.path.join(DIR_WRITE, f'Figure_3D.{filetype}'),
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1)

##### Figure 3 combined

In [ ]:
dpi = 300
filetype = 'pdf'

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

xticklabels = ['1$^\mathrm{st}$ quarter', '2$^\mathrm{nd}$ quarter', 
               '3$^\mathrm{rd}$ quarter', '4$^\mathrm{th}$ quarter']

fig, ax = plt.subplots(2,2, figsize = [12,6])
fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, hspace=0.4, wspace=0.2)

# Figure 3A

ax[0,0].boxplot(spon_mand_longitudinal['PIP_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax[0,0].set_ylim(0, 1.4)
ax[0,0].set_ylabel('PIP$_\mathrm{PS}$ / PIP$_\mathrm{VG}$')
ax[0,0].set_xticklabels(xticklabels)

ax[0,0].text(2.5, 1.2, 'p < 0.01', fontsize = 10, horizontalalignment='center')

# Figure 3B

ax[0,1].boxplot(spon_mand_longitudinal['VT_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax[0,1].set_ylim(0, 1.4)
ax[0,1].set_ylabel('VT$_\mathrm{PS}$ / VT$_\mathrm{VG}$')
ax[0,1].set_xticklabels(xticklabels)

ax[0,1].text(2.5, 1.2, 'p < 0.01', fontsize = 10, horizontalalignment='center')

# Figure 3C

ax[1,0].boxplot(spon_mand_longitudinal['RR_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax[1,0].set_ylim(0, 2.8)
ax[1,0].set_ylabel('RR$_\mathrm{PS}$ / RR$_\mathrm{VG}$')
ax[1,0].set_xticklabels(xticklabels)

ax[1,0].text(2.5, 2.3, 'p < 0.0001', fontsize = 10, horizontalalignment='center')

# Figure 3D

ax[1,1].boxplot(spon_mand_longitudinal['MVe_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax[1,1].set_ylim(0, 2.1)
ax[1,1].set_ylabel('MV$_\mathrm{PS}$ / MV$_\mathrm{VG}$')
ax[1,1].set_xticklabels(xticklabels)

ax[1,1].text(2.5, 1.7, 'p < 0.0001', fontsize = 10, horizontalalignment='center')


fig.text(0.04, 0.9, 'A', fontsize = 16); fig.text(0.49, 0.9, 'B', fontsize = 16)
fig.text(0.04, 0.48, 'C', fontsize = 16); fig.text(0.49, 0.48, 'D', fontsize = 16)

fig.savefig(os.path.join(DIR_WRITE, f'Figure_3.{filetype}'), dpi = dpi, bbox_inches='tight');

In [ ]:
dpi = 300
filetype = 'jpg'

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

xticklabels = ['1$^\mathrm{st}$ quarter', '2$^\mathrm{nd}$ quarter', 
               '3$^\mathrm{rd}$ quarter', '4$^\mathrm{th}$ quarter']

fig, ax = plt.subplots(2,2, figsize = [12,6])
fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, hspace=0.4, wspace=0.2)

# Figure 3A

ax[0,0].boxplot(spon_mand_longitudinal['PIP_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax[0,0].set_ylim(0, 1.4)
ax[0,0].set_ylabel('PIPspon / PIPmand')
ax[0,0].set_xticklabels(xticklabels)

ax[0,0].text(2.5, 1.2, 'p < 0.01', fontsize = 10, horizontalalignment='center')

# Figure 3B

ax[0,1].boxplot(spon_mand_longitudinal['VT_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax[0,1].set_ylim(0, 1.4)
ax[0,1].set_ylabel('VTspon / VTmand')
ax[0,1].set_xticklabels(xticklabels)

ax[0,1].text(2.5, 1.2, 'p < 0.01', fontsize = 10, horizontalalignment='center')

# Figure 3C

ax[1,0].boxplot(spon_mand_longitudinal['RR_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax[1,0].set_ylim(0, 2.8)
ax[1,0].set_ylabel('RRspon / RRmand')
ax[1,0].set_xticklabels(xticklabels)

ax[1,0].text(2.5, 2.3, 'p < 0.0001', fontsize = 10, horizontalalignment='center')

# Figure 3D

ax[1,1].boxplot(spon_mand_longitudinal['MVe_spon_mand'], whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

ax[1,1].set_ylim(0, 2.1)
ax[1,1].set_ylabel('MVspon / MVmand')
ax[1,1].set_xticklabels(xticklabels)

ax[1,1].text(2.5, 1.7, 'p < 0.0001', fontsize = 10, horizontalalignment='center')


fig.text(0.04, 0.9, 'A', fontsize = 16); fig.text(0.49, 0.9, 'B', fontsize = 16)
fig.text(0.04, 0.48, 'C', fontsize = 16); fig.text(0.49, 0.48, 'D', fontsize = 16)

fig.savefig(os.path.join(DIR_WRITE, f'Figure_3_alt.{filetype}'), dpi = dpi, bbox_inches='tight');

### 20. E-Figures for the paper

#### E-Figure 2

In [ ]:
dpi=300
filetype = 'pdf'

patient = 'LVD002'
data = data_1min_mean_inrange[patient]

fig, axes = plt.subplots(nrows=2, ncols=1, figsize = [12, 8])
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, hspace=0.1, wspace=None)
  
# Supplementary Figure 2A
data['PIP [mbar]'].plot(ax = axes[0], color = 'red',  marker='o', linestyle='', markersize=1, x_compat = True)
data['PIP_spon [mbar]'].plot(ax = axes[0],color = 'green', marker='o', linestyle='', markersize=1, x_compat = True)
        
axes[0].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[0].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
axes[0].set_ylim(0, axes[0].get_ylim()[1]+15)
axes[0].set_xlabel('', size = 12, color = 'black')
axes[0].set_ylabel('mbar', size = 12, color = 'black')
axes[0].set_title('',  size = 12, color = 'black')
axes[0].legend(['PIP$_\mathrm{VG}$', 'PIP$_\mathrm{PS}$',], ncol = 2, fontsize = 12, loc = 2, markerscale=6)
#axes[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
axes[0].tick_params(bottom=False, labelbottom=False)
   
# Supplementary Figure 2B
data['VTspon [mL/kg]'].plot(ax = axes[1], color = 'green', linestyle='', marker='o', markersize=1, x_compat = True)
data['VTmand [mL/kg]'].plot(ax = axes[1], color = 'red', linestyle='', marker='o', markersize=1, x_compat = True)
data['VT_set [mL/kg]'].plot(ax = axes[1], color = 'black', linewidth=2, linestyle='dashed', x_compat = True)
            
axes[1].xaxis.set_major_formatter(majorFmt)
plt.setp(axes[1].xaxis.get_majorticklabels(), rotation=0, fontsize = 12, horizontalalignment = 'center')
axes[1].set_ylim(-2, 25)
axes[1].set_xlabel('Dates (day/month)', size = 12, color = 'black')
axes[1].set_ylabel('mL/kg', size = 12, color = 'black')
axes[1].set_title('',  size = 12, color = 'black')
axes[1].legend(['VT$_\mathrm{PS}$', 'VT$_\mathrm{VG}$', 'VT$_\mathrm{SET}$'], 
               ncol = 3, fontsize = 12, loc = 2, markerscale=6)
#axes[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')

fig.text(0.06, 0.90, 'A', fontsize = 16); fig.text(0.06, 0.54, 'B', fontsize = 16)
        
fig.savefig(os.path.join(DIR_WRITE, f'E_Figure_2.{filetype}'), 
    dpi = dpi, format = filetype, bbox_inches='tight', pad_inches=0.1,)

#### E-Figure 3

In [ ]:
dpi=300
filetype = 'pdf'

medianprops = {'color': 'black', 'linewidth': 1}
meanpointprops = {'marker':'D', 'markersize' : 4, 'markeredgecolor':'black', 'markerfacecolor':'black'}
boxprops = {'color': 'black',  'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
flierprops = {'color': 'black', 'marker': '.'}

fig, axes = plt.subplots(nrows=2, ncols=1, figsize = [12, 8])
fig.subplots_adjust(left=0.1, bottom=0.1, right=None, top=None, hspace=0.1, wspace=None)

# Supplementary Figure 3A
data = list(data_1min_mean_inrange_all['PIP_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

axes[0].boxplot(data_for_graph, whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

axes[0].set_ylim(0, 1.1)
axes[0].hlines(0.66, *axes[0].get_xlim(), color='black', linestyle='--', linewidth=1)
axes[0].set_xticklabels(xticklabels)
axes[0].set_ylabel('PIP$_\mathrm{PS}$ / PIP$_\mathrm{VG}$')
axes[0].tick_params(bottom=False, labelbottom=False)

# Supplementary Figure 3B
data = list(data_1min_mean_inrange_all['VT_spon_mand'].dropna().groupby(level=1))
data_for_graph = [item[1] for item in data]
xticklabels = [item[0] for item in data]

axes[1].boxplot(data_for_graph, whis = [5,95], showfliers = False,
    showmeans = False, medianprops=medianprops, meanprops = meanpointprops, 
    boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops, )

axes[1].set_ylim(0, 2)
axes[1].set_xticklabels(xticklabels)
axes[1].set_ylabel('VT$_\mathrm{PS}$ / VT$_\mathrm{VG}$')

fig.text(0.04, 0.90, 'A', fontsize = 16); fig.text(0.04, 0.48, 'B', fontsize = 16)
    
fig.savefig(os.path.join(DIR_WRITE, f'E_Figure_3.{filetype}'), dpi = dpi, format = filetype, 
    bbox_inches='tight', pad_inches=0.1)